Dataset disponible sur: https://www.kaggle.com/datasets/muhammadhananasghar/human-emotions-datasethes/code?select=Emotions+Dataset

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import sklearn
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import datetime
import io
import os
import random
import time
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, MaxPool2D, Dense, InputLayer, Flatten, BatchNormalization, Input, Dropout, RandomFlip, RandomRotation, RandomContrast, Resizing, Rescaling, GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Embedding, LayerNormalization, MultiHeadAttention, Permute
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives, Precision, Recall, AUC, binary_accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, CSVLogger, EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import L2, L1
from tensorflow.train import BytesList, FloatList, Int64List, Example, Features, Feature
from tensorboard.plugins.hparams import api as hp
import wandb
from wandb.keras import WandbCallback

## Wandb

In [ ]:
!wandb login

In [ ]:
wandb.init(project="Emotion-Detection", entity="fredlicombeau")

In [ ]:
wandb.config = {
    "BATCH_SIZE": 32,
    "IM_SIZE": 256,
    "LEARNING_RATE": 1e-3,
    "N_EPOCHS": 20,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASSES": 3,
    "PATCH_SIZE": 16,
    "PROJ_DIM": 768,
    "CLASS_NAMES": ["angry", "happy", "sad"],
}
CONFIGURATION = wandb.config

## Data Management

In [ ]:
# CONFIGURATION = {
#     "BATCH_SIZE": 32,
#     "IM_SIZE": 256,
#     "LEARNING_RATE": 1e-3,
#     "N_EPOCHS": 20,
#     "DROPOUT_RATE": 0.0,
#     "REGULARIZATION_RATE": 0.0,
#     "N_FILTERS": 6,
#     "KERNEL_SIZE": 3,
#     "N_STRIDES": 1,
#     "POOL_SIZE": 2,
#     "N_DENSE_1": 1024,
#     "N_DENSE_2": 128,
#     "NUM_CLASSES": 3,
#     "PATCH_SIZE": 16,
#     "PROJ_DIM": 768,
#     "CLASS_NAMES": ["angry", "happy", "sad"],
# }

In [ ]:
train_directory = "./dataset/Emotions Dataset/Emotions Dataset/train"
val_directory = "./dataset/Emotions Dataset/Emotions Dataset/test"
CLASS_NAMES = ["angry", "happy", "sad"]

## Dataset Loading

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CLASS_NAMES,
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99
)

In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    labels='inferred',
    label_mode='categorical',
    class_names=CLASS_NAMES,
    color_mode='rgb',
    batch_size=1,
    image_size=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    shuffle=True,
    seed=99
)

In [ ]:
# for i in val_dataset.take(1):
#     print(i)

## Dataset Visualization

In [ ]:
plt.figure(figsize=(12, 12))

for images, labels in train_dataset.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i+1)
        plt.imshow(images[i]/255.)
        plt.title(CONFIGURATION["CLASS_NAMES"][tf.argmax(labels[i], axis = 0).numpy()])
        plt.axis("off")

## Data Augmentation

In [ ]:
augment_layers = tf.keras.Sequential([
    RandomRotation(factor=(-0.025, 0.025)),
    RandomFlip(mode='horizontal'),
    RandomContrast(factor=0.1)
])

In [ ]:
def augment_layer(image, label):
    return augment_layers(image, training=True), label

### Cutmix Augmentation

In [ ]:
def box(lamda):
    r_x = tf.cast(tfp.distributions.Uniform(0, CONFIGURATION['IM_SIZE']).sample(1)[0], dtype=tf.int32)
    r_y = tf.cast(tfp.distributions.Uniform(0, CONFIGURATION['IM_SIZE']).sample(1)[0], dtype=tf.int32)

    r_w = tf.cast(CONFIGURATION['IM_SIZE']*tf.math.sqrt(1-lamda), dtype=tf.int32)
    r_h = tf.cast(CONFIGURATION['IM_SIZE']*tf.math.sqrt(1-lamda), dtype=tf.int32)
    
    r_x = tf.clip_by_value(r_x - r_w//2, 0, CONFIGURATION['IM_SIZE'])
    r_y = tf.clip_by_value(r_y - r_h//2, 0, CONFIGURATION['IM_SIZE'])

    x_b_r = tf.clip_by_value(r_x + r_w//2, 0, CONFIGURATION['IM_SIZE'])
    y_b_r = tf.clip_by_value(r_y + r_h//2, 0, CONFIGURATION['IM_SIZE'])

    r_w = x_b_r - r_x
    if r_w == 0:
        r_w = 1

    r_h = y_b_r - r_y
    if r_h == 0:
        r_h = 1
    
    return r_y, r_x, r_h, r_w

In [ ]:
def cutmix(train_dataset_1, train_dataset_2):
    (image_1, label_1), (image_2, label_2) = train_dataset_1, train_dataset_2

    lamda = tfp.distributions.Beta(0.2, 0.2)
    lamda = lamda.sample(1)[0]
    
    r_y, r_x, r_h, r_w = box(lamda)

    crop_2 = tf.image.crop_to_bounding_box(image_2, r_y, r_x, r_h, r_w)
    pad_2 = tf.image.pad_to_bounding_box(crop_2, r_y, r_x, CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE'])

    crop_1 = tf.image.crop_to_bounding_box(image_1, r_y, r_x, r_h, r_w)
    pad_1 = tf.image.pad_to_bounding_box(crop_1, r_y, r_x, CONFIGURATION['IM_SIZE'], CONFIGURATION['IM_SIZE'])

    image = image_1 - pad_1 + pad_2

    lamda = tf.cast(1 - (r_w*r_h)/(CONFIGURATION['IM_SIZE']*CONFIGURATION['IM_SIZE']), dtype=tf.float32)
    label = lamda * tf.cast(label_1, dtype=tf.float32) + (1-lamda) * tf.cast(label_2, dtype=tf.float32)

    return image, label

In [ ]:
train_dataset_1 = train_dataset.map(augment_layer, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset_2 = train_dataset.map(augment_layer, num_parallel_calls=tf.data.AUTOTUNE)

mixed_dataset = tf.data.Dataset.zip((train_dataset_1, train_dataset_2))

## Dataset Preparation

In [ ]:
training_dataset = (
    mixed_dataset
    .map(cutmix, num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
validation_dataset = (
    val_dataset
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
resize_rescale_layers = tf.keras.Sequential([
    Resizing(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]),
    Rescaling(1./255),
])

In [ ]:
training_dataset

In [ ]:
validation_dataset

### TFRecords

In [ ]:
training_dataset = (
    training_dataset
    .unbatch()
)

In [ ]:
validation_dataset = (
    validation_dataset
    .unbatch()
)

In [ ]:
training_dataset

In [ ]:
validation_dataset

In [ ]:
def create_example(image, label):
    bytes_feature = Feature(
        bytes_list=BytesList(value=[image])
    )
    int_feature = Feature(
        int64_list=Int64List(value=[label])
    )

    example = Example(
        features=Features(feature={
            'images': bytes_feature,
            'labels': int_feature,
        })
    )

    return example.SerializeToString()

In [ ]:
NUM_SHARDS = 10
PATH = 'tfrecords/shard_{:02d}.tfrecord'

In [ ]:
def encode_image(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.uint8)
    image = tf.io.encode_jpeg(image)
    return image, tf.argmax(label)

In [ ]:
encoded_dataset = (
    training_dataset
    .map(encode_image)
)

In [ ]:
for shard_number in range(NUM_SHARDS):
    sharded_dataset = (
        encoded_dataset
        .shard(NUM_SHARDS, shard_number)
        .as_numpy_iterator()
    )

    with tf.io.TFRecordWriter(PATH.format(shard_number)) as file_writer:
        for encoded_image, encoded_label in sharded_dataset:
            example = create_example(encoded_image, encoded_label)
            file_writer.write(example)

In [ ]:
recons_dataset = tf.data.TFRecordDataset(filenames=[PATH.format(p) for p in range(NUM_SHARDS)])

In [ ]:
def parse_tfrecords(example):
    feature_description = {
        'images': tf.io.FixedLenFeature([], tf.string),
        'labels': tf.io.FixedLenFeature([], tf.int64),
    }
    
    example = tf.io.parse_single_example(example, feature_description)
    example["images"] = tf.io.decode_jpeg(example["images"], channels=3)

    return example["images"], example["labels"]

In [ ]:
parsed_dataset = (
    recons_dataset
    .map(parse_tfrecords)
    .batch(CONFIGURATION["BATCH_SIZE"])
    .prefetch(tf.data.AUTOTUNE)
)

In [ ]:
parsed_dataset

## Modeling

### LeNet

In [ ]:
lenet_model = tf.keras.Sequential([
    InputLayer(input_shape = (None, None, 3),),

    resize_rescale_layers,

    Conv2D(filters=CONFIGURATION["N_FILTERS"], kernel_size=CONFIGURATION["KERNEL_SIZE"], strides=CONFIGURATION["N_STRIDES"], padding='valid', activation="relu", kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"])),
    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGURATION["POOL_SIZE"], strides=CONFIGURATION["N_STRIDES"]*2),
    Dropout(rate=CONFIGURATION["DROPOUT_RATE"]),

    Conv2D(filters=CONFIGURATION["N_FILTERS"]*2 + 4, kernel_size=CONFIGURATION["KERNEL_SIZE"], strides=CONFIGURATION["N_STRIDES"], padding='valid', activation="relu", kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"])),
    BatchNormalization(),
    MaxPool2D(pool_size=CONFIGURATION["POOL_SIZE"], strides=CONFIGURATION["N_STRIDES"]*2),

    Flatten(),

    Dense(CONFIGURATION["N_DENSE_1"], activation="relu", kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"])),
    BatchNormalization(),
    Dropout(rate=CONFIGURATION["DROPOUT_RATE"]),

    Dense(CONFIGURATION["N_DENSE_2"], activation="relu", kernel_regularizer=L2(CONFIGURATION["REGULARIZATION_RATE"])),
    BatchNormalization(),

    Dense(CONFIGURATION["NUM_CLASSES"], activation="softmax")
])

lenet_model.summary()

### ResNet34

In [ ]:
class CustomConv2D(Layer):
    def __init__(self, n_filters, kernel_size, n_strides, padding='valid'):
        super(CustomConv2D, self).__init__(name='custom_conv2d')

        self.conv = Conv2D(
            filters = n_filters,
            kernel_size = kernel_size,
            activation = 'relu',
            strides = n_strides,
            padding = padding
        )

        self.batch_norm = BatchNormalization()
    
    def call(self, x, training=True):
        x = self.conv(x)
        x = self.batch_norm(x, training)

        return x

In [ ]:
class ResidualBlock(Layer):
    def __init__(self, n_channels, n_strides=1):
        super(ResidualBlock, self).__init__(name='res_block')

        self.dotted = (n_strides != 1)

        self.custom_conv_1 = CustomConv2D(n_channels, 3, n_strides, padding='same')
        self.custom_conv_2 = CustomConv2D(n_channels, 3, 1, padding='same')

        self.activation = Activation('relu')

        if self.dotted:
            self.custom_conv_3 = CustomConv2D(n_channels, 1, n_strides)
        
    def call(self, input, training):
        x = self.custom_conv_1(input, training)
        x = self.custom_conv_2(x, training)

        if self.dotted:
            x_add = self.custom_conv_3(input, training)
            x_add = Add()([x, x_add])
        else:
            x_add = Add()([x, input])

        return self.activation(x_add)

In [ ]:
class ResNet34(Model):
    def __init__(self):
        super(ResNet34, self).__init__(name='resnet_34')

        self.conv_1 = CustomConv2D(64, 7, 2, padding='same')
        self.max_pool = MaxPooling2D(3, 2)

        self.conv_2_1 = ResidualBlock(64)
        self.conv_2_2 = ResidualBlock(64)
        self.conv_2_3 = ResidualBlock(64)

        self.conv_3_1 = ResidualBlock(128, 2)
        self.conv_3_2 = ResidualBlock(128)
        self.conv_3_3 = ResidualBlock(128)
        self.conv_3_4 = ResidualBlock(128)

        self.conv_4_1 = ResidualBlock(256, 2)
        self.conv_4_2 = ResidualBlock(256)
        self.conv_4_3 = ResidualBlock(256)
        self.conv_4_4 = ResidualBlock(256)
        self.conv_4_5 = ResidualBlock(256)
        self.conv_4_6 = ResidualBlock(256)

        self.conv_5_1 = ResidualBlock(512, 2)
        self.conv_5_2 = ResidualBlock(512)
        self.conv_5_3 = ResidualBlock(512)

        self.global_pool = GlobalAveragePooling2D()

        self.fc_3 = Dense(CONFIGURATION["NUM_CLASSES"], activation='softmax')
    
    def call(self, x, training=True):
        x = self.conv_1(x)
        x = self.max_pool(x)
        
        x = self.conv_2_1(x, training)
        x = self.conv_2_2(x, training)
        x = self.conv_2_3(x, training)

        x = self.conv_3_1(x, training)
        x = self.conv_3_2(x, training)
        x = self.conv_3_3(x, training)
        x = self.conv_3_4(x, training)

        x = self.conv_4_1(x, training)
        x = self.conv_4_2(x, training)
        x = self.conv_4_3(x, training)
        x = self.conv_4_4(x, training)
        x = self.conv_4_5(x, training)
        x = self.conv_4_6(x, training)

        x = self.conv_5_1(x, training)
        x = self.conv_5_2(x, training)
        x = self.conv_5_3(x, training)

        x = self.global_pool(x)

        return self.fc_3(x)

In [ ]:
resnet_34 = ResNet34()
resnet_34(tf.zeros([1, 256, 256, 3]), training=False)
resnet_34.summary()

### Transfer Learning with EfficientNet

In [ ]:
backbone = tf.keras.applications.efficientnet.EfficientNetB4(
    include_top=False,
    weights='imagenet',
    input_shape=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3),
)

In [ ]:
backbone.trainable = False

In [ ]:
pretrained_model = tf.keras.Sequential([
    Input(shape=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3)),
    backbone,
    GlobalAveragePooling2D(),
    Dense(CONFIGURATION["N_DENSE_1"], activation="relu"),
    BatchNormalization(),
    Dense(CONFIGURATION["N_DENSE_2"], activation="relu"),
    Dense(CONFIGURATION["NUM_CLASSES"], activation="softmax"),
])
pretrained_model.summary()

### FineTuning EfficientNet

In [ ]:
backbone.trainable = False # Lancer une fois en False puis en True sans lancer les 2 cases suivantes. Relancer le compile avec le learning rate /100.

In [ ]:
input = Input(shape=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3)),

x = backbone(input, training=False)

x = GlobalAveragePooling2D()(x)
x = Dense(CONFIGURATION["N_DENSE_1"], activation="relu")(x)
x = BatchNormalization()(x)
x = Dense(CONFIGURATION["N_DENSE_2"], activation="relu")(x)
output = Dense(CONFIGURATION["NUM_CLASSES"], activation="softmax")(x)

finetuned_model = Model(input, output)

In [ ]:
finetuned_model.summary()

### Vision Transformers

#### Patch Encoder

In [ ]:
test_image = cv2.imread("./dataset/Emotions Dataset/Emotions Dataset/test/sad/123731.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))

In [ ]:
patches = tf.image.extract_patches(images=tf.expand_dims(test_image, axis=0),
                                   sizes=[1, CONFIGURATION["PATCH_SIZE"], CONFIGURATION["PATCH_SIZE"], 1],
                                   strides=[1, CONFIGURATION["PATCH_SIZE"], CONFIGURATION["PATCH_SIZE"], 1],
                                   rates=[1, 1, 1, 1],
                                   padding='VALID')

In [ ]:
print(patches.shape)
patches = tf.reshape(patches, (patches.shape[0], -1, 768))
print(patches.shape)

In [ ]:
plt.figure(figsize=(8, 8))

for i in range(patches.shape[1]):
    ax = plt.subplot(16, 16, i+1)
    plt.imshow(tf.reshape(patches[0, i, :], (16, 16, 3)))
    plt.axis("off")

In [ ]:
class PatchEncoder(Layer):
    def __init__(self, N_PATCHES, HIDDEN_SIZE):
        super(PatchEncoder, self).__init__(name='patch_encoder')

        self.linear_projection = Dense(HIDDEN_SIZE)
        self.positional_embedding = Embedding(N_PATCHES, HIDDEN_SIZE)
        self.N_PATCHES = N_PATCHES
    
    def call(self, x):
        patches = tf.image.extract_patches(images=x,
                                   sizes=[1, CONFIGURATION["PATCH_SIZE"], CONFIGURATION["PATCH_SIZE"], 1],
                                   strides=[1, CONFIGURATION["PATCH_SIZE"], CONFIGURATION["PATCH_SIZE"], 1],
                                   rates=[1, 1, 1, 1],
                                   padding='VALID')
        
        patches = tf.reshape(patches, (tf.shape(patches)[0], -1, patches.shape[-1]))
        embedding_input = tf.range(start=0, limit=self.N_PATCHES, delta=1)
        output = self.linear_projection(patches) + self.positional_embedding(embedding_input)
        
        return output

In [ ]:
patch_enc = PatchEncoder(256, 768)
patch_enc(tf.zeros([32, 256, 256, 3]))

#### Transformer Encoder

In [ ]:
class TransformerEncoder(Layer):
    def __init__(self, N_HEADS, HIDDEN_SIZE):
        super(TransformerEncoder, self).__init__(name='transformer_encoder')

        self.layer_norm_1 = LayerNormalization()
        self.layer_norm_2 = LayerNormalization()
        
        self.multi_head_att = MultiHeadAttention(N_HEADS, HIDDEN_SIZE)

        self.dense_1 = Dense(HIDDEN_SIZE, activation=tf.nn.gelu)
        self.dense_2 = Dense(HIDDEN_SIZE, activation=tf.nn.gelu)
    
    def call(self, input):
        x_1 = self.layer_norm_1(input)
        x_1 = self.multi_head_att(x_1, x_1)

        x_1 = Add()([x_1, input])

        x_2 = self.layer_norm_2(x_1)
        x_2 = self.dense_1(x_2)
        output = self.dense_2(x_2)
        output = Add()([output, x_1])
        
        return output

In [ ]:
trans_enc = TransformerEncoder(8, 768)
trans_enc(tf.zeros([1, 256, 768]))

#### ViT Model

In [ ]:
class ViT(Model):
    def __init__(self, N_HEADS, HIDDEN_SIZE, N_PATCHES, N_LAYERS, N_DENSE_UNITS):
        super(ViT, self).__init__(name='vision_transformer')

        self.N_LAYERS = N_LAYERS
        self.patch_encoder = PatchEncoder(N_PATCHES, HIDDEN_SIZE)
        self.trans_encoders = [TransformerEncoder(N_HEADS, HIDDEN_SIZE) for _ in range(N_LAYERS)]
        self.dense_1 = Dense(N_DENSE_UNITS, tf.nn.gelu)
        self.dense_2 = Dense(N_DENSE_UNITS, tf.nn.gelu)
        self.dense_3 = Dense(CONFIGURATION["NUM_CLASSES"], activation='softmax')
    
    def call(self, input, training=True):
        x = self.patch_encoder(input)

        for i in range(self.N_LAYERS):
            x = self.trans_encoders[i](x)
        
        x = Flatten()(x)
        x = self.dense_1(x)
        x = self.dense_2(x)
        
        return self.dense_3(x)

In [ ]:
vit = ViT(4, 768, 256, 2, 128)
vit(tf.zeros([32, 256, 256, 3]))

In [ ]:
vit.summary()

### HuggingFace ViT

#### Training

In [ ]:
resize_rescale_hf = tf.keras.Sequential([
       Resizing(224, 224),
       Rescaling(1./255),
       Permute((3,1,2))              
])

In [ ]:
from transformers import ViTFeatureExtractor, TFViTModel, ViTConfig, ViTModel

base_model = TFViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

inputs = Input(shape = (256,256,3))
x = resize_rescale_hf(inputs)
x = base_model.vit(x)[0][:,0,:]
output = Dense(CONFIGURATION["NUM_CLASSES"], activation = 'softmax')(x)

hf_model = tf.keras.Model(inputs=inputs, outputs=output)

In [ ]:
test_image = cv2.imread("./dataset/Emotions Dataset/Emotions Dataset/train/happy/387249.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))

In [ ]:
hf_model(tf.expand_dims(test_image, axis=0))

In [ ]:
hf_model.summary()

#### Get Attention Maps

In [ ]:
from transformers import ViTFeatureExtractor, TFViTModel, ViTConfig

configuration = ViTConfig()
configuration.output_attentions = True

base_model = TFViTModel.from_pretrained(
    pretrained_model_name_or_path = "google/vit-base-patch16-224-in21k",
    config = configuration,
)
inputs = Input(shape = (256,256,3))
x = resize_rescale_hf(inputs)
x = base_model.vit(x)['attentions']

model = tf.keras.Model(inputs=inputs, outputs=x)

## Training

### Callback

In [ ]:
checkpoint_callback = ModelCheckpoint(
    'best_weights', 
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True
)

In [ ]:
class LogConfMatrix(Callback):
  def on_epoch_end(self, epoch, logs):
    predicted = []
    labels = []

    for im, label in validation_dataset:
      predicted.append(hf_model(im))
      labels.append(label.numpy())

    pred = np.concatenate([np.argmax(predicted[:-1], axis = -1).flatten(), np.argmax(predicted[-1], axis = -1).flatten()])
    lab = np.concatenate([np.argmax(labels[:-1], axis = -1).flatten(), np.argmax(labels[-1], axis = -1).flatten()])
    
    cm = wandb.plot.confusion_matrix(
        y_true=lab,
        preds=pred,
        class_names=CONFIGURATION["CLASS_NAMES"])
        
    wandb.log({"conf_mat": cm})

In [ ]:
class LogResultsTable(Callback):
  def on_epoch_end(self, epoch, logs):
    
    columns=["image", "Predicted", "Label"]
    
    val_table = wandb.Table(columns = columns)

    
    for im, label in validation_dataset.take(25):

      pred = CONFIGURATION["CLASS_NAMES"][tf.argmax(hf_model(im), axis = -1).numpy()[0]]
      label = CONFIGURATION["CLASS_NAMES"][tf.argmax(label, axis = -1).numpy()[0]]

      row = [wandb.Image(im), pred, label]
      
      val_table.add_data(*row)

      
    wandb.log({"Model Results" : val_table})

### Train

In [ ]:
loss_function = CategoricalCrossentropy()
# loss_function = SparseCategoricalCrossentropy()

In [ ]:
metrics = [CategoricalAccuracy(name="accuracy"), TopKCategoricalAccuracy(k=2, name="top_k_accuracy")]

In [ ]:
# quant_aware_model.compile(optimizer=Adam(learning_rate=CONFIGURATION['LEARNING_RATE']),
#                     loss=loss_function,
#                     metrics=metrics)
# hf_model.compile(optimizer=Adam(learning_rate=5e-5), # 5e-5
#                     loss=loss_function,
#                     metrics=metrics)
vit.compile(optimizer=Adam(learning_rate=CONFIGURATION['LEARNING_RATE']),
                    loss=loss_function,
                    metrics=metrics)
# finetuned_model.compile(optimizer=Adam(learning_rate=CONFIGURATION['LEARNING_RATE']), # /100
#                     loss=loss_function,
#                     metrics=metrics)
# pretrained_model.compile(optimizer=Adam(learning_rate=CONFIGURATION['LEARNING_RATE']),
#                     loss=loss_function,
#                     metrics=metrics)
# resnet_34.compile(optimizer=Adam(learning_rate=CONFIGURATION['LEARNING_RATE']*10),
#                     loss=loss_function,
#                     metrics=metrics)
# lenet_model.compile(optimizer=Adam(learning_rate=CONFIGURATION['LEARNING_RATE']),
#                     loss=loss_function,
#                     metrics=metrics)

In [ ]:
my_devices = tf.config.experimental.list_physical_devices(device_type='GPU')
tf.config.experimental.set_visible_devices(devices= my_devices, device_type='GPU')

In [ ]:
# history = quant_aware_model.fit(training_dataset.take(20),
#                           validation_data=validation_dataset, 
#                           epochs=1,
#                           verbose=1)
# history = hf_model.fit(training_dataset.take(20),
#                        validation_data = validation_dataset,
#                        epochs = 3,
#                        verbose = 1,
#                        callbacks = [WandbCallback(), LogConfMatrix(), LogResultsTable()]) # Trop coûteux en mémoire de GPU -> 9Go. A tester sur Google Collab.
history = vit.fit(training_dataset.take(20),
                          validation_data=validation_dataset, 
                          epochs=1,
                          verbose=1)
# history = finetuned_model.fit(training_dataset.take(10),
#                           validation_data=validation_dataset, 
#                           epochs=CONFIGURATION["N_EPOCHS"],
#                           verbose=1)
# history = pretrained_model.fit(training_dataset.take(10),
#                           validation_data=validation_dataset, 
#                           epochs=CONFIGURATION["N_EPOCHS"],
#                           verbose=1)
# history = resnet_34.fit(training_dataset,
#                           validation_data=validation_dataset, 
#                           epochs=CONFIGURATION["N_EPOCHS"]*3,
#                           verbose=1,
#                           callbacks=[checkpoint_callback])
# history = lenet_model.fit(training_dataset,
#                           validation_data=validation_dataset,
#                           epochs=CONFIGURATION["N_EPOCHS"],
#                           verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy', 'val_accuracy'])
plt.show()

## Evaluation

In [ ]:
pretrained_model.load_weights('best_weights')

In [ ]:
pretrained_model.evaluate(validation_dataset)
# resnet_34.evaluate(validation_dataset)
# lenet_model.evaluate(validation_dataset)

## Testing

In [ ]:
test_image = cv2.imread("./dataset/Emotions Dataset/Emotions Dataset/test/happy/520857.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))

im = tf.constant(test_image, dtype=tf.float32)

im = tf.expand_dims(im, axis=0)

print(pretrained_model(im))
print(CLASS_NAMES[tf.argmax(pretrained_model(im), axis=-1).numpy()[0]])

In [ ]:
test_image = cv2.imread("./dataset/Emotions Dataset/Emotions Dataset/test/sad/105639.jpg")

im = tf.constant(test_image, dtype=tf.float32)

im = tf.expand_dims(im, axis=0)

print(resnet_34(im))
print(CLASS_NAMES[tf.argmax(resnet_34(im), axis=-1).numpy()[0]])

In [ ]:
test_image = cv2.imread("./dataset/Lilian.png")

im = tf.constant(test_image, dtype=tf.float32)

im = tf.expand_dims(im, axis=0)

print(resnet_34(im))
print(CLASS_NAMES[tf.argmax(resnet_34(im), axis=-1).numpy()[0]])

In [ ]:
plt.figure(figsize=(12, 12))

for images, labels in train_dataset.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i+1)
        plt.imshow(images[i]/255.)
        plt.title("True Label: " + CONFIGURATION["CLASS_NAMES"][tf.argmax(labels[i], axis = -1).numpy()]  + "\nPredicted Label: " 
              + CONFIGURATION["CLASS_NAMES"][int(tf.argmax(pretrained_model(tf.expand_dims(images[i], axis = 0)), axis =-1).numpy()[0])])
        plt.axis("off")

## Visualization

### Feature Map Visualization

In [ ]:
vgg_backbone = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3)
)

In [ ]:
vgg_backbone.summary()

In [ ]:
def is_conv(layer_name):
    if 'conv' in layer_name:
        return True
    return False

In [ ]:
feature_maps = [layer.output for layer in vgg_backbone.layers[1:] if is_conv(layer.name)]
feature_map_model = Model(
    inputs=vgg_backbone.input,
    outputs=feature_maps
)

feature_map_model.summary()

In [ ]:
test_image = cv2.imread("./dataset/Emotions Dataset/Emotions Dataset/test/happy/520857.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"]))

im = tf.constant(test_image, dtype=tf.float32)
im = tf.expand_dims(im, axis=0)

f_maps = feature_map_model.predict(im)

In [ ]:
for i in range(len(f_maps)):
    print(f_maps[i].shape)

In [ ]:
for i in range(len(f_maps)):
    plt.figure(figsize=(256, 256))
    f_size = f_maps[i].shape[1]
    n_channels = f_maps[i].shape[-1]
    joint_maps = np.ones((f_size, f_size*n_channels))
    
    axs = plt.subplot(len(f_maps), 1, i+1)

    for j in range(n_channels):
        joint_maps[:, f_size*j:f_size*(j+1)] = f_maps[i][..., j]

    plt.imshow(joint_maps[:, 0:512])
    plt.axis("off")

### GradCam

In [ ]:
backbone = tf.keras.applications.efficientnet.EfficientNetB5(
    include_top=False,
    weights='imagenet',
    input_shape=(CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3),
)
backbone.trainable=False

In [ ]:
x = backbone.output

x = GlobalAveragePooling2D()(x)
x = Dense( CONFIGURATION["N_DENSE_1"], activation = "relu")(x)
x = Dense( CONFIGURATION["N_DENSE_2"], activation = "relu")(x)
output = Dense( CONFIGURATION["NUM_CLASSES"], activation = "softmax")(x)

pretrained_model = Model(backbone.inputs, output)
pretrained_model.summary()

In [ ]:
img_path = "./dataset/Emotions Dataset/Emotions Dataset/train/happy/202291.jpg"

In [ ]:
test_image = cv2.imread(img_path)
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))
im = tf.constant(test_image, dtype = tf.float32)
img_array = tf.expand_dims(im, axis = 0)
print(img_array.shape)

In [ ]:
preds = pretrained_model.predict(img_array)

In [ ]:
print(preds)

In [ ]:
last_conv_layer_name = "top_activation"
last_conv_layer = pretrained_model.get_layer(last_conv_layer_name)
last_conv_layer_model = Model(pretrained_model.inputs, last_conv_layer.output)

In [ ]:
classifier_layer_names = [
    "global_average_pooling2d",
    "dense",
    "dense_1",
    "dense_2"
]

In [ ]:
classifier_input = Input(shape=(8, 8, 2048))
x = classifier_input 
for layer_name in classifier_layer_names:
    x = pretrained_model.get_layer(layer_name)(x)
classifier_model = Model(classifier_input, x)

In [ ]:
with tf.GradientTape() as tape:
    last_conv_layer_output = last_conv_layer_model(img_array)
    preds = classifier_model(last_conv_layer_output)
    top_pred_index = tf.argmax(preds[0])
    top_class_channel = preds[:, top_pred_index]

grads = tape.gradient(top_class_channel, last_conv_layer_output)

In [ ]:
grads.shape

In [ ]:
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2)).numpy()

In [ ]:
pooled_grads.shape

In [ ]:
last_conv_layer_output = last_conv_layer_output.numpy()[0]
for i in range(pooled_grads.shape[0]):
    last_conv_layer_output[:, :, i] *= pooled_grads[i]

In [ ]:
last_conv_layer_output.shape

In [ ]:
heatmap = np.sum(last_conv_layer_output, axis=-1)

In [ ]:
heatmap = tf.nn.relu(heatmap)
plt.matshow(heatmap)

In [ ]:
resized_heatmap = cv2.resize(np.array(heatmap), (256, 256))
plt.matshow(resized_heatmap*2555+img_array[0,:,:,0]/255)

## Exporting to Onnx format

In [ ]:
hf_model.save('vit_finetuned')

In [ ]:
hf_model.save('vit_finetuned.h5')

### Conversion

#### From Tensorflow SavedModel

In [ ]:
!python -m tf2onnx.convert --saved-model vit_finetuned/ --output vit_onnx.onnx

#### From Keras Model

In [ ]:
import tf2onnx
import onnxruntime as rt

spec = (tf.TensorSpec(
    (None, CONFIGURATION["IM_SIZE"], CONFIGURATION["IM_SIZE"], 3),
    tf.float32, name="input"),)

output_path = "vit_keras.onnx"

model_proto, _ = tf2onnx.convert.from_keras(hf_model, input_signature=spec, opset=13, output_path=output_path)

### Inference

In [ ]:
test_image = cv2.imread("/dataset/Emotions Dataset/Emotions Dataset/test/sad/123731.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))
im = test_image.astype(np.float32)

im = np.expand_dims(im, axis = 0)

In [ ]:
N_PREDICTIONS = 10

### Benchmarking Onnx

In [ ]:
providers=['CPUExecutionProvider']

In [ ]:
# m = rt.InferenceSession("./vit_keras.onnx", providers=providers)
m = rt.InferenceSession("./vit_quantized.onnx", providers=providers)
t1 = time.time()

for _ in range(N_PREDICTIONS):
  onnx_pred = m.run(['dense'], {"input": im})
print("Time for a single Prediction", (time.time() - t1)/N_PREDICTIONS)

In [ ]:
print(onnx_pred)

### Benchmarking TF

In [ ]:
t1 = time.time()
for _ in range(N_PREDICTIONS):
  hf_model(im)
print("Time for a single Prediction", (time.time() - t1)/N_PREDICTIONS)

In [ ]:
# tf, gpu = 0.15s
# tf, cpu = 0.8s
# tf_size = 1000MB

# onnx, cpu = 0.5s
# onnx, gpu = 0.025s
# onnx_size = 328MB

# onnx_quantized, cpu = 0.4s
# onnx_quantized, gpu = 0.3s
# onnx_quantized_size = 83MB

### Quantization with Onnx

In [ ]:
import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

In [ ]:
# model_fp32 = 'vit_keras.onnx'
# model_quant = 'vit_quantized.onnx'
model_fp32 = 'eff_keras.onnx'
model_quant = 'eff_quantized.onnx'

quantized_model = quantize_dynamic(model_fp32, model_quant, weight_type = QuantType.QUInt8)

In [ ]:
def accuracy(model):
  total, acc = 0,0
  for im, label in validation_dataset:
    onnx_pred = model.run(output_names, {"input": np.array(im)})

    if(int(np.argmax(onnx_pred, axis = -1)[0][0]) == int(np.argmax(label, axis = -1)[0])):
      acc += 1

    total += 1
  return acc/total

In [ ]:
m = rt.InferenceSession("vit_keras.onnx", providers=providers)
m_q = rt.InferenceSession("vit_quantized.onnx", providers=providers)
print(accuracy(m_q))
print(accuracy(m))

In [ ]:
img_path = "/dataset/Emotions Dataset/Emotions Dataset/test/happy/553112.jpg"
test_image = cv2.imread(img_path)
print(test_image.shape)
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))
im = np.float32(test_image)
img_array = np.expand_dims(im, axis = 0)
print(img_array.shape)

### Quantization with Tensorflow

In [ ]:
import tensorflow_model_optimization as tfmot

#### Quantization Aware Training

In [ ]:
backbone,

x = GlobalAveragePooling2D()(backbone.output)
x = Dense(CONFIGURATION["N_DENSE_1"], activation="relu")(x)
x = BatchNormalization()(x)
x = Dense(CONFIGURATION["N_DENSE_2"], activation="relu")(x)
output = Dense(CONFIGURATION["NUM_CLASSES"], activation="softmax")(x)

pretrained_func_model = tf.keras.Model(inputs=backbone.input, outputs=output)

pretrained_func_model.summary()

In [ ]:
pretrained_model.summary()

In [ ]:
#quant_aware = tfmot.quantization.keras.quantize_model(lenet_model)

In [ ]:
def apply_quantization_to_conv(layer):
    if "conv" in layer.name:
        return tfmot.quantization.keras.quantize_annotate_layer(layer)
    return layer

In [ ]:
quant_aware_eff = tf.keras.models.clone_model(
    pretrained_func_model, clone_function=apply_quantization_to_conv
)

In [ ]:
quant_aware_eff.summary()

In [ ]:
quant_aware_model = tfmot.quantization.keras.quantize_apply(quant_aware_eff)
quant_aware_model.summary()

#### Post Training Quantization

In [ ]:
pretrained_model.load_weights("eff_keras.h5")

In [ ]:
pretrained_model.evaluate(validation_dataset)

In [ ]:
def representative_data_gen():
    for input_value, j in training_dataset.take(20):
        yield [input_value]

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(pretrained_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen

In [ ]:
tflite_model = converter.convert()

In [ ]:
import pathlib

tflite_models_dir = pathlib.Path("/quantized_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"eff_model.tflite"
tflite_model_file.write_bytes(tflite_model)

In [ ]:
pretrained_model.save("eff_model.h5")

#### TFLite Runtime

In [ ]:
!pip3 install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

In [ ]:
import tflite_runtime as tflite

In [ ]:
test_image = cv2.imread("/dataset/Emotions Dataset/Emotions Dataset/train/happy/148266.jpg")
test_image = cv2.resize(test_image, (CONFIGURATION["IM_SIZE"] ,CONFIGURATION["IM_SIZE"]))
test_image = np.expand_dims(test_image, axis = 0)

#print(CONFIGURATION['CLASS_NAMES'][tf.argmax(pretrained_model(im), axis = -1).numpy()[0]])

In [ ]:
interpreter = tflite.Interpreter(model_path="eff_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
# test_image = im.numpy().astype(input_details["dtype"])

interpreter.set_tensor(input_details["index"], test_image)
interpreter.invoke()

output = interpreter.get_tensor(output_details["index"])[0]

In [ ]:
print(CONFIGURATION['CLASS_NAMES'][np.argmax(output)])

#### Accuracy of Quantized Model

In [ ]:
def accuracy(model_path):
    total, correct = 0,0
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    for im, label in validation_dataset:
        
        test_image = im.numpy().astype(input_details["dtype"])

        interpreter.set_tensor(input_details["index"], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details["index"])[0]

        if(int(np.argmax(output)) == int(np.argmax(label, axis = -1)[0])):
            correct += 1

        total += 1

    return correct/total

In [ ]:
accuracy("eff_model.tflite")

## Confusion Matrix

In [ ]:
predicted = []
labels = []

for im, label in validation_dataset:
    predicted.append(pretrained_model(im))
    labels.append(label.numpy())

In [ ]:
print(np.concatenate([np.argmax(labels[:-1], axis = -1).flatten(), np.argmax(labels[-1], axis = -1).flatten()]))
print(np.concatenate([np.argmax(predicted[:-1], axis = -1).flatten(), np.argmax(predicted[-1], axis = -1).flatten()]))

In [ ]:
pred = np.concatenate([np.argmax(labels[:-1], axis = -1).flatten(), np.argmax(labels[-1], axis = -1).flatten()])
lab = np.concatenate([np.argmax(predicted[:-1], axis = -1).flatten(), np.argmax(predicted[-1], axis = -1).flatten()])

In [ ]:
cm = confusion_matrix(lab, pred)
print(cm)

plt.figure(figsize=(5, 5))
sns.heatmap(cm, annot=True)
plt.title('Confusion matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')